In [8]:
%%sh

# The name of our algorithm
repository=sklearn
tag=naive_bayes
algorithm_name=naive_bayes
#cd container

#chmod +x decision_trees/train
#chmod +x decision_trees/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${repository}:${tag}"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${repository}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${repository}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

# On a SageMaker Notebook Instance, the docker daemon may need to be restarted in order
# to detect your network configuration correctly.  (This is a known issue.)
if [ -d "/home/ec2-user/SageMaker" ]; then
  sudo service docker restart
fi

#docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
The push refers to a repository [328115522647.dkr.ecr.us-east-1.amazonaws.com/sklearn]
b0b10e843dee: Preparing
cc4edab414c8: Preparing
035205ca1db6: Preparing
e2f5d0e1c167: Preparing
6dc84ce2c971: Preparing
2f5b0990636a: Preparing
c9748fbf541d: Preparing
b3968bc26fbd: Preparing
aa4e47c45116: Preparing
788ce2310e2f: Preparing
b3968bc26fbd: Waiting
aa4e47c45116: Waiting
2f5b0990636a: Waiting
c9748fbf541d: Waiting
6dc84ce2c971: Layer already exists
035205ca1db6: Layer already exists
b0b10e843dee: Layer already exists
e2f5d0e1c167: Layer already exists
cc4edab414c8: Layer already exists
2f5b0990636a: Layer already exists
c9748fbf541d: Layer already exists
b3968bc26fbd: Layer already exists
788ce2310e2f: Layer already exists
aa4e47c45116: Layer already exists
naive_bayes: digest: sha256:225fb152c2cd35481f7bd42a845d4f9affec97064bd5588d9cabeef08fad85fb size: 2411


WARNING! Using --password via the CLI is insecure. Use --password-stdin.


In [4]:
%%sh
docker images

REPOSITORY                                             TAG                 IMAGE ID            CREATED             SIZE
naive_bayes                                            latest              3f6530187cec        2 days ago          1.1GB
karan4080/sklearn                                      latest              7aaa4a63a989        2 days ago          1.1GB
karan4080/sklearn                                      <none>              1f68307c4f27        2 days ago          1.1GB
karan4080/sklearn_ubuntu                               latest              743b43c60a32        3 days ago          991MB
328115522647.dkr.ecr.us-east-1.amazonaws.com/sklearn   ubuntu              20c44cd7596f        2 weeks ago         123MB
ubuntu                                                 latest              20c44cd7596f        2 weeks ago         123MB


In [11]:
prefix = 'scikit-byo-iris'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

ValueError: The current AWS identity is not a role: arn:aws:iam::328115522647:user/KaranNadagoudar, therefore it cannot be used as a SageMaker execution role